In [1]:
# Install OS dependencies.  This only needs to be run once for each new notebook instance.
!pip install PyAthena

  Using cached tenacity-7.0.0-py2.py3-none-any.whl (23 kB)


In [4]:
from pyathena import connect
#from pyathena.util import as_pandas
#from __future__ import print_function

# Import libraries
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import boto3
from botocore.client import ClientError
# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline


s3 = boto3.resource('s3')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'aws-athena-query-results-'+account_id+'-'+region

try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp').cursor()


# The Glue database name of your MIMIC-III parquet data
gluedatabase="mimiciii"

# location of the queries to generate aline specific materialized views
aline_path = './'

# location of the queries to generate materialized views from the MIMIC code repository
concepts_path = './concepts/'

Creating bucket aws-athena-query-results-530388612227-us-east-1


In [9]:
conn = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp')

In [7]:
query = '''
SELECT * FROM mimiciii.noteevents LIMIT 5
'''

In [10]:
pd.read_sql(query,conn)

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,2004671,24291,148229,2163-01-19,2163-01-19 18:39:00,2163-01-19 18:41:00,Nursing/other,Report,15522,None,Nursing NICU note Addendum.\nMother in this ev...
1,2004672,24291,148229,2163-01-20,2163-01-20 06:45:00,2163-01-20 07:00:00,Nursing/other,Report,16024,None,NPN 1900-0700\n\n\n1. FEN: TF=130cc/k/day B...
2,2005856,24416,197351,2117-11-20,2117-11-20 01:48:00,2117-11-20 02:07:00,Nursing/other,Report,21222,None,Admission Note\nOb- [**Month (only) 910**]\nPe...
3,2005857,24416,197351,2117-11-20,2117-11-20 02:14:00,2117-11-20 02:17:00,Nursing/other,Report,16024,None,Nursing Triage Note\nInfant is a full term mal...
4,2002347,24131,117291,2174-10-11,2174-10-11 08:08:00,2174-10-11 08:12:00,Nursing/other,Report,20888,None,[**Location (un) 369**]/NEON DOL 5 CGA 34 [**5...
